In [14]:
from aif360.algorithms.inprocessing import PrejudiceRemover,AdversarialDebiasing
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import joblib
import re
import sys
sys.path.insert(0, '../')
import numpy as np

# Datasets
from aif360.datasets import MEPSDataset19
from aif360.datasets import MEPSDataset20
from aif360.datasets import MEPSDataset21
from aif360.datasets import GermanDataset
# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler


from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric



In [15]:
dataset_orig_panel19 = MEPSDataset19()

In [16]:
dataset_orig_panel19_train, dataset_orig_panel19_val = dataset_orig_panel19.split([0.7], shuffle=True)

In [17]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]]

In [18]:
def test(dataset, model, thresh, unprivileged_groups, privileged_groups):
    y_val_pred_prob = model.predict(dataset).scores
    pos_ind = 0  # Assuming scores are for the favorable outcome

    # Ensure y_val_pred_prob is two-dimensional
    if y_val_pred_prob.ndim == 1:
        # If y_val_pred_prob is one-dimensional, use it directly
        y_val_pred = (y_val_pred_prob > thresh).astype(int)
    else:
        # Use pos_ind to index the second dimension
        y_val_pred = (y_val_pred_prob[:, pos_ind] > thresh).astype(int)

    # Continue with the evaluation using y_val_pred
    # Calculate accuracy, fairness metrics, etc., based on y_val_pred

    return {
        'accuracy': None,  # Replace with actual accuracy calculation
        'fairness_metric': None  # Replace with actual fairness metric calculation
    }


In [19]:
features = dataset_orig_panel19_train.features
label = dataset_orig_panel19_train.labels
feature_names = dataset_orig_panel19_train.feature_names
dataset_orig_panel19_train_df = pd.DataFrame(features, columns=feature_names)
dataset_orig_panel19_train_df['RACE2']=dataset_orig_panel19_train_df['RACE']
dataset_orig_panel19_train_df.drop('RACE',axis=1,inplace=True)
dataset_orig_panel19_train_df.rename(columns={'RACE2': 'RACE'},inplace=True)
dataset_orig_panel19_train_df['label'] = label

In [20]:
dataset_orig_panel19_train_df

,AGE,PCS42,MCS42,K6SUM42,REGION=1,REGION=2,REGION=3,REGION=4,SEX=1,SEX=2,...,POVCAT=1,POVCAT=2,POVCAT=3,POVCAT=4,POVCAT=5,INSCOV=1,INSCOV=2,INSCOV=3,RACE,label
0,57.0,54.80,60.12,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,29.0,57.47,54.69,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,53.0,52.40,62.66,2.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,55.0,-1.00,-1.00,-1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
4,51.0,-1.00,-1.00,-1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11076,35.0,53.03,43.53,4.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
11077,17.0,-1.00,-1.00,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
11078,58.0,-1.00,-1.00,-1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11079,36.0,56.71,62.39,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [21]:
from aif360.datasets import StandardDataset

# Define which column is the label and which one is the protected attribute
label_name = 'label'  # or whatever the name of your label column is
protected_attribute_name = 'RACE'  # or your protected attribute column name

# Create the aif360 dataset
aif360_dataset = StandardDataset(
    df=dataset_orig_panel19_train_df,
    label_name=label_name,
    favorable_classes=[1],  # assuming 1 is the favorable class
    protected_attribute_names=[protected_attribute_name],
    privileged_classes=[[1.0]],  # assuming 'White' is the privileged class
)


In [22]:
aif360_dataset_train,aif360_dataset_test=aif360_dataset.split([0.7], shuffle=True)

In [23]:
aif360_dataset_train

               instance weights features                                 \
                                                                          
                                     AGE  PCS42  MCS42 K6SUM42 REGION=1   
instance names                                                            
3965                        1.0     66.0  40.09  48.40    -9.0      0.0   
1752                        1.0     28.0  55.66  50.01     2.0      0.0   
284                         1.0      5.0  -1.00  -1.00    -1.0      0.0   
1558                        1.0     51.0  45.64  62.40     1.0      0.0   
4408                        1.0     76.0  16.34  53.12     0.0      0.0   
...                         ...      ...    ...    ...     ...      ...   
5186                        1.0     52.0  53.42  44.83     3.0      1.0   
7196                        1.0     14.0  -1.00  -1.00    -1.0      1.0   
3134                        1.0     72.0  57.78  50.82     0.0      0.0   
2473                     

In [24]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

pr=PrejudiceRemover(sensitive_attr='RACE', eta=25.0)
dset_scaled_trn = aif360_dataset_train.copy()
dset_scaled_trn.features = scaler.fit_transform(dset_scaled_trn.features)

pr_fitted = pr.fit(dset_scaled_trn)
print(pr_fitted)
accs = []
thresholds = np.linspace(0.01, 0.5, 10)

dset_val = aif360_dataset_test.copy()
dset_val.features = scaler.transform(dset_val.features)

##################### STEP 1 TRAINING WITH INPROCESSING #####################
pr_pred_prob = pr_fitted.predict(dset_val).scores
print(pr_pred_prob)
##################### STEP 2 PICKING THRESHOLD WITH VALIDATION DATA #####################
for threshold in thresholds:
    dset_val_pred = dset_val.copy()
    dset_val_pred.labels = (pr_pred_prob[:, 0] > threshold).astype(np.float64)

    metric = ClassificationMetric(
                dset_val, dset_val_pred,
                unprivileged_groups = unprivileged_groups,
                privileged_groups=privileged_groups)
    accs.append((metric.true_positive_rate() + metric.true_negative_rate()) / 2)


pr_val_best_idx = np.argmax(accs)
best_threshold = thresholds[pr_val_best_idx]

##################### STEP 3 TEST DATA #####################
dset_tst = aif360_dataset_test.copy()
dset_tst.features = scaler.transform(dset_tst.features)

pr_pred_prob = pr_fitted.predict(dset_tst).scores


dset_tst_pred = dset_tst.copy()
dset_tst_pred.labels = (pr_pred_prob[:, 0] > best_threshold).astype(np.float64)

metric = ClassificationMetric(
            dset_tst, dset_tst_pred,
            unprivileged_groups = unprivileged_groups,
            privileged_groups   = privileged_groups)
test_acc = (metric.true_positive_rate() + metric.true_negative_rate()) / 2 ## no built in balanced error rate
test_disp_impact = metric.disparate_impact()

print("Testing accuracy with ETA %0.2f = %0.2f\n Disparate impact %0.2f" % (25.0, test_acc, test_disp_impact))

STDOUT: b'Model loaded.\r\nData shape: (3325, 139)\r\nFeatures shape: (3325, 138), Labels shape: (3325,)\r\nPredictions shape: (3325, 2)\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\nWriting prediction for sample 3324: class 0\r\n'
STDERR: b''
(3325, 5)
[[0.45902625]
 [0.22868268]
 [0.03929404]
 ...
 [0.03253647]
 [0.55638827]
 [0.45576

In [25]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

# Now you can use tf.Session like in TensorFlow 1.x
session = tf.compat.v1.Session()

debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          num_epochs=10,
                          debias=True,
                          sess=session)

debiased_model.fit(dataset_orig_panel19_train)

#fair = get_fair_metrics_and_plot(data_orig_test, debiased_model, plot=False, model_aif=True)
data_pred = debiased_model.predict(dataset_orig_panel19_val)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

epoch 0; iter: 0; batch classifier loss: 1.724557; batch adversarial loss: 0.708830
epoch 1; iter: 0; batch classifier loss: 0.604499; batch adversarial loss: 0.681954
epoch 2; iter: 0; batch classifier loss: 0.321243; batch adversarial loss: 0.682153
epoch 3; iter: 0; batch classifier loss: 0.300242; batch adversarial loss: 0.670921
epoch 4; iter: 0; batch classifier loss: 0.346055; batch adversarial loss: 0.662858
epoch 5; iter: 0; batch classifier loss: 0.332033; batch adversarial loss: 0.650315
epoch 6; iter: 0; batch classifier loss: 0.281160; batch adversarial loss: 0.655163
epoch 7; iter: 0; batch classifier loss: 0.349158; batch adversarial loss: 0.624533
epoch 8; iter: 0; batch classifier loss: 0.324309; batch adversarial loss: 0.624527
epoch 9; iter: 0; batch classifier loss: 0.308793; batch adversarial loss: 0.611027


In [26]:
metric_transf_panel19_train = BinaryLabelDatasetMetric(
data_pred,
unprivileged_groups=unprivileged_groups,
privileged_groups=privileged_groups)
explainer_transf_panel19_train = MetricTextExplainer(metric_transf_panel19_train)
test_results_ad=[explainer_transf_panel19_train.mean_difference()
               ,explainer_transf_panel19_train.consistency()
               ,explainer_transf_panel19_train.statistical_parity_difference()
               ,explainer_transf_panel19_train.disparate_impact()]

In [27]:
test_results_ad

['Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.061699987002589565',
 'Consistency (Zemel, et al. 2013): [0.95712782]',
 'Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.061699987002589565',
 'Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.4779928304847973']

In [28]:

from aif360.algorithms.inprocessing import MetaFairClassifier
from aif360.algorithms.inprocessing import GerryFairClassifier

In [29]:
MC = MetaFairClassifier(tau=0, sensitive_attr="RACE", type="fdr")

In [30]:
mc_model = MC.fit(dataset_orig_panel19_train)

In [31]:
dataset_transf_panel19_train_MC = mc_model.predict(dataset_orig_panel19_train)

c:\Users\srinivas\anaconda3\Lib\site-packages\aif360\algorithms\inprocessing\celisMeta\FalseDiscovery.py:31: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
c:\Users\srinivas\anaconda3\Lib\site-packages\aif360\algorithms\inprocessing\celisMeta\FalseDiscovery.py:32: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
c:\Users\srinivas\anaconda3\Lib\site-packages\aif360\algorithms\inprocessing\celisMeta\FalseDiscovery.py:33: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total
c:\Users\srinivas\anaconda3\Lib\site-packages\aif360\algorithms\inprocessing\celisMeta\FalseDiscovery.py:35: RuntimeWarning: invalid value encountered in divide
  probc_m1_0 = prob_m1_0 / total
c:\Users\srinivas\anaconda3\Lib\site-packages\aif360\algorithms\inprocessing\celisMeta\FalseDiscovery.py:36: RuntimeWarning: invalid value encountered in divide
  probc_m1_1 = prob_m1_1 / tota

In [32]:
dataset_transf_panel19_test_MC = mc_model.predict(dataset_orig_panel19_val)

In [33]:
classified_metric_bias_test = BinaryLabelDatasetMetric(dataset_transf_panel19_test_MC,
                                                   unprivileged_groups=unprivileged_groups,
                                                   privileged_groups=privileged_groups)
explainer_transf_panel19_train = MetricTextExplainer(metric_transf_panel19_train)
test_results_mc=[explainer_transf_panel19_train.mean_difference()
               ,explainer_transf_panel19_train.consistency()
               ,explainer_transf_panel19_train.statistical_parity_difference()
               ,explainer_transf_panel19_train.disparate_impact()]

In [34]:
test_results_mc

['Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.061699987002589565',
 'Consistency (Zemel, et al. 2013): [0.95712782]',
 'Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.061699987002589565',
 'Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.4779928304847973']

In [35]:
GC = GerryFairClassifier(C= 100, printflag=True, gamma=.005, fairness_def='FP',
             max_iters=10, heatmapflag=False)

In [36]:
# fit method
gfc_model = GC.fit(dataset_orig_panel19_train, early_termination=True)

# predict method. If threshold in (0, 1) produces binary predictions
dataset_yhat = gfc_model.predict(dataset_orig_panel19_train, threshold=False)

iteration: 1, error: 0.1330204855157477, fairness violation: 0.006406795146862778, violated group size: 0.27019222091868966
iteration: 2, error: 0.15003158559696778, fairness violation: 0.003234917463099615, violated group size: 0.5602382456456998
iteration: 3, error: 0.15552146316517765, fairness violation: 0.0021776249018452265, violated group size: 0.27019222091868966
iteration: 4, error: 0.15808591282375237, fairness violation: 0.001656858593635089, violated group size: 0.27019222091868966
iteration: 5, error: 0.15953433805613212, fairness violation: 0.00133852634139932, violated group size: 0.5602382456456998
iteration: 6, error: 0.16042475107541435, fairness violation: 0.0011465989153143603, violated group size: 0.27019222091868966
iteration: 7, error: 0.1609834078925316, fairness violation: 0.0010182053575703398, violated group size: 0.5602382456456998
iteration: 8, error: 0.16136855879433265, fairness violation: 0.0009219101892623255, violated group size: 0.27019222091868966
it

In [37]:
# auditing 
from aif360.algorithms.inprocessing.gerryfair.clean import array_to_tuple

gerry_metric = BinaryLabelDatasetMetric(dataset_orig_panel19_train)
gamma_disparity = gerry_metric.rich_subgroup(array_to_tuple(dataset_yhat.labels), 'FP')
print(gamma_disparity)

0.0008537786390468316


In [38]:
from sklearn import svm
from sklearn import tree
from sklearn.kernel_ridge import KernelRidge
from sklearn import linear_model
import pickle

In [39]:
# set to 10 iterations for fast running of notebook - set >= 1000 when running real experiments
# tests learning with different hypothesis classes
pareto_iters = 10
def multiple_classifiers_pareto(dataset, gamma_list=[0.002, 0.005, 0.01], save_results=True, iters=pareto_iters):

    ln_predictor = linear_model.LinearRegression()
    svm_predictor = svm.LinearSVR()
    tree_predictor = tree.DecisionTreeRegressor(max_depth=3)
    kernel_predictor = KernelRidge(alpha=1.0, gamma=1.0, kernel='rbf')
    predictor_dict = {'Linear': {'predictor': ln_predictor, 'iters': iters},
                       'SVR': {'predictor': svm_predictor, 'iters': iters},
                       'Tree': {'predictor': tree_predictor, 'iters': iters},
                       'Kernel': {'predictor': kernel_predictor, 'iters': iters}}
    #predictor_dict = {'Linear': {'predictor': ln_predictor, 'iters': iters}}

    results_dict = {}

    for pred in predictor_dict:
        print('Curr Predictor: {}'.format(pred))
        predictor = predictor_dict[pred]['predictor']
        max_iters = predictor_dict[pred]['iters']
        fair_clf = GerryFairClassifier(C=100, printflag=True, gamma=1, predictor=predictor, max_iters=max_iters)
        fair_clf.printflag = False
        fair_clf.max_iters=max_iters
        errors, fp_violations, fn_violations = fair_clf.pareto(dataset, gamma_list)
        results_dict[pred] = {'errors': errors, 'fp_violations': fp_violations, 'fn_violations': fn_violations}
    if save_results:
        print('path:'+'results_dict_' + str(gamma_list) + '_gammas' + str(gamma_list) + '.pkl', 'wb')
        pickle.dump(results_dict, open('results_dict_' + str(gamma_list) + '_gammas' + str(gamma_list) + '.pkl', 'wb'))

multiple_classifiers_pareto(dataset_orig_panel19_train)

Curr Predictor: Linear
Curr Predictor: SVR


c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `

Curr Predictor: Tree
Curr Predictor: Kernel
path:results_dict_[0.002, 0.005, 0.01]_gammas[0.002, 0.005, 0.01].pkl wb


: 